In [ ]:
# Fix pybel import path
try:
    import sys
    sys.modules['pybel'] = __import__('openbabel', globals(), locals(), ['pybel']).pybel
except Exception:
    pass

In [ ]:
import ipywidgets as ipw
from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import CodQueryWidget, SmilesWidget, StructureExamplesWidget
from aiidalab_widgets_base import OptimadeQueryWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import WizardAppWidget

from aiidalab_qe.process import WorkChainSelector
from aiidalab_qe.steps import SubmitQeAppWorkChainStep
from aiidalab_qe.steps import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_qe.structures import StructureSelectionStep


OptimadeQueryWidget.title = "OPTIMADE"  # monkeypatch


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        CodQueryWidget(title="COD"),
        StructureBrowserWidget(title="AiiDA database"),
        OptimadeQueryWidget(embedded=True),
        SmilesWidget(title="SMILES"),  # requires OpenBabel! 
        StructureExamplesWidget(
            title="From Examples",
            examples=[
                ("Silicon", "miscellaneous/structures/Si.xyz"),
                ("Silicon oxide", "miscellaneous/structures/SiO2.xyz"),
            ]),
    ],
    node_class='StructureData',
)
structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True
submit_qe_app_work_chain_step = SubmitQeAppWorkChainStep()
submit_qe_app_work_chain_step.auto_advance = True
view_qe_app_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_qe_app_work_chain_step, 'input_structure'))
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (view_qe_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_qe_app_work_chain_step),
        ('Status & Results', view_qe_app_work_chain_status_and_results_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_qe_app_work_chain_step.process = process
        
work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

welcome_message = ipw.HTML(
    """
    <div style="padding-top: 0px; padding-bottom: 0px; line-height: 140%;">
    <h3>Welcome to the AiiDAlab Quantum ESPRESSO app! 👋</h3>

    The Quantum ESPRESSO app (or QEapp for short) is a graphical interface for calculating material properties based on the density-functional theory (DFT) as implemented in the <a href="https://www.quantum-espresso.org/" target="_blank">Quantum ESPRESSO software suite</a>.
    Each of the properties is calculated by workflows powered by the <a href="https://www.aiida.net/" target="_bland"> AiiDA engine</a>, and maintained in the <a href="https://aiida-quantumespresso.readthedocs.io/en/latest/" target="_blank"> Quantum ESPRESSO plugin</a> for AiiDA.

    <p style="padding-top: 10px; padding-bottom: 10px;">The QEapp allows you to calculate material properties in a simple 3-step process:</p>

    <ol style="list-style-type:none">
      <li style="padding-top: 2px; padding-bottom: 2px;">🔍 <b>Step 1:</b> Select the structure you want to run.</li>
      <li style="padding-top: 2px; padding-bottom: 2px;">⚙️ <b>Step 2:</b> Select the properties you are interested in.</li> 
      <li style="padding-top: 2px; padding-bottom: 2px;">🚀 <b>Step 3:</b> Submit your workflow!</li> 
    </ol>

    <p style="padding-top: 5px; padding-bottom: 5px;">New users can go straight to the first step and select their structure. Once you've already calculated some properties, you can select the corresponding workflow using the dropdown below.</p>

    <p style="padding-top: 5px; padding-bottom: 5px;"><em>Happy computing! 🎉</em></p>
    </div>
    """
)
app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

display(welcome_message, app_with_work_chain_selector)